In [ ]:
import asyncio

async def consumer(name, queue, semaphore):
    async with semaphore:
        to_consume = await queue.get()
        print(f'consumer {name} consuming {to_consume}')
        asyncio.sleep(1)
        l = len(to_consume)
        return [to_consume[:l//2], to_consume[l//2:]]
    
async def producer():
    consumed = set()
    queue = asyncio.Queue()
    semaphore = asyncio.Semaphore(5)
    await queue.put('abcdefghijklmn')
    workers = []
    for _ in range(5):
        worker = asyncio.create_task(consumer(queue, semaphore, consumed))
        workers.append(worker)

In [1]:
[][:-1]

[]